# Bronchus Segmentation

This example demonstrated how to use the bronchial tree segmentation algorithm used in:

***W. Ghandourh, J. Dowling, P. Chlap et al., “Medical Dosimetry Assessing tumor centrality in lung stereotactic ablative body radiotherapy ( SABR ): the effects of variations in bronchial tree delineation and potential for automated methods,” vol. 46, pp. 94–101, 2021, doi: [10.1016/j.meddos.2020.09.004](https://doi.org/10.1016/j.meddos.2020.09.004).***

## Import Modules

In [ ]:
try:
    import platipy
except:
    !pip install platipy
    import platipy

import SimpleITK as sitk

%matplotlib inline

from platipy.imaging.tests.data import get_lung_nifti
from platipy.imaging.projects.bronchus.run import run_bronchus_segmentation
from platipy.imaging import ImageVisualiser
from platipy.imaging.label.utils import get_com

## Download Test Data

This will download lung cancer patient CT scans, with contours of several structures.
This only has to be done once - if it is ran again don't worry, the same data will not be downloaded again!

In [ ]:
data_path = get_lung_nifti()

## Load Test Image

Read in the image we want to automatically segment


In [ ]:
test_pat_path = data_path.joinpath("LCTSC-Test-S1-201")
test_image = sitk.ReadImage(str(test_pat_path.joinpath("IMAGES/LCTSC_TEST_S1_201_0_CT_0.nii.gz")))

## Run Auto-segmentation

This will take some time, and will print updates along the way


In [ ]:
auto_structures = run_bronchus_segmentation(test_image)

## Save segmentations
Optionally write the automatic segmentations to disk


In [ ]:
output_directory = test_pat_path.joinpath("SEGMENTATIONS")
output_directory.mkdir(exist_ok=True)

for struct_name in list(auto_structures.keys()):
    sitk.WriteImage(auto_structures[struct_name], str(output_directory.joinpath(f"{struct_name}.nii.gz")))

print(f"Segmentations saved to: {output_directory}")



## Visualise Segmentations

Next, we can generate a nice figure to check what the segmentation looks like

The first step here is to convert SimpleITK images to NumPy arrays

In [ ]:
vis = ImageVisualiser(test_image, cut=get_com(auto_structures["Auto_Bronchus"]))

vis.add_contour({struct: auto_structures[struct] for struct in auto_structures.keys()})

fig = vis.show()